In [13]:
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import OneHotEncoder




In [8]:
df = pd.read_csv('../../raw_data/flights.csv',nrows=500000)

In [9]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])

In [11]:
#_________________________________________________________
# Function that convert the 'HHMM' string to datetime.time
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure
#_____________________________________________________________________
# Function that combines a date and time to produce a datetime.datetime
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
#_______________________________________________________________________________
# Function that combine two columns of the dataframe to create a datetime format
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)

In [14]:
df['SCHEDULED_DEPARTURE'] = create_flight_time(df, 'SCHEDULED_DEPARTURE')
df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].apply(format_heure)
df['SCHEDULED_ARRIVAL'] = df['SCHEDULED_ARRIVAL'].apply(format_heure)
df['ARRIVAL_TIME'] = df['ARRIVAL_TIME'].apply(format_heure)
#__________________________________________________________________________
df.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME',
             'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,2015-01-01 00:05:00,04:30:00,23:54:00,04:08:00,-11.0,-22.0
1,2015-01-01 00:10:00,07:50:00,00:02:00,07:41:00,-8.0,-9.0
2,2015-01-01 00:20:00,08:06:00,00:18:00,08:11:00,-2.0,5.0
3,2015-01-01 00:20:00,08:05:00,00:15:00,07:56:00,-5.0,-9.0
4,2015-01-01 00:25:00,03:20:00,00:24:00,02:59:00,-1.0,-21.0
5,2015-01-01 00:25:00,06:02:00,00:20:00,06:10:00,-5.0,8.0


In [15]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


,variable,missing values,filling factor (%)
0,CANCELLATION_REASON,483176,3.3648
1,LATE_AIRCRAFT_DELAY,397701,20.4598
2,AIRLINE_DELAY,397701,20.4598
3,SECURITY_DELAY,397701,20.4598
4,AIR_SYSTEM_DELAY,397701,20.4598
5,WEATHER_DELAY,397701,20.4598
6,ELAPSED_TIME,17948,96.4104
7,AIR_TIME,17948,96.4104
8,ARRIVAL_DELAY,17948,96.4104
9,WHEELS_ON,17159,96.5682


In [16]:
df[df['WEATHER_DELAY']>0]['CANCELLED'].value_counts()


CANCELLED
0    7417
Name: count, dtype: int64

In [17]:
df[df['SECURITY_DELAY']>0]['CANCELLED'].value_counts()


CANCELLED
0    297
Name: count, dtype: int64

In [18]:
df[df['AIRLINE_DELAY']>0]['CANCELLED'].value_counts()


CANCELLED
0    53966
Name: count, dtype: int64

In [19]:
features_prelim = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
                     'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'CANCELLED']
cancelation_df = df[features_prelim].copy()

In [20]:
cancelation_df.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED
0,1,1,4,AS,ANC,SEA,205.0,1448,04:30:00,0
1,1,1,4,AA,LAX,PBI,280.0,2330,07:50:00,0
2,1,1,4,US,SFO,CLT,286.0,2296,08:06:00,0
3,1,1,4,AA,LAX,MIA,285.0,2342,08:05:00,0
4,1,1,4,AS,SEA,ANC,235.0,1448,03:20:00,0


In [29]:
cancelation_df.to_csv('../../raw_data/cancelation_df.csv', index=False)

In [21]:
print(f"The unique values for 'Airlines' are {cancelation_df.AIRLINE.unique()}")

The unique values for 'Airlines' are ['AS' 'AA' 'US' 'DL' 'NK' 'UA' 'HA' 'B6' 'OO' 'EV' 'MQ' 'F9' 'WN' 'VX']


In [22]:
cancelation_df.AIRLINE.value_counts()

AIRLINE
WN    106253
DL     68555
EV     52965
OO     51184
AA     46950
UA     40873
US     35591
MQ     31896
B6     23062
AS     14149
NK      9324
F9      7291
HA      6858
VX      5049
Name: count, dtype: int64

In [23]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse = False) 

# Fit encoder
ohe.fit(cancelation_df[['AIRLINE']]) 

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")


The categories detected by the OneHotEncoder are [array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA',
       'US', 'VX', 'WN'], dtype=object)]


/Users/diego/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [24]:
# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "Street" column
cancelation_df[ohe.get_feature_names_out()] = ohe.transform(cancelation_df[['AIRLINE']])

# Drop the column "Street" which has been encoded
cancelation_df.drop(columns = ["AIRLINE"], inplace = True)

# Show the dataset
cancelation_df.head(3)

The column names for the encoded values are ['AIRLINE_AA' 'AIRLINE_AS' 'AIRLINE_B6' 'AIRLINE_DL' 'AIRLINE_EV'
 'AIRLINE_F9' 'AIRLINE_HA' 'AIRLINE_MQ' 'AIRLINE_NK' 'AIRLINE_OO'
 'AIRLINE_UA' 'AIRLINE_US' 'AIRLINE_VX' 'AIRLINE_WN']


,MONTH,DAY,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,AIRLINE_AA,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
0,1,1,4,ANC,SEA,205.0,1448,04:30:00,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,4,LAX,PBI,280.0,2330,07:50:00,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,4,SFO,CLT,286.0,2296,08:06:00,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# FILTRANDO EL DATASET

### PARA ESTO UTILIZAREMOS EL DATASET COMPLETO (5M DE DATOS)

In [27]:
df_completo = pd.read_csv('../../raw_data/flights.csv')#,nrows=500000)

/var/folders/wd/v2y3d4d948j0h2gc56s15nzr0000gn/T/ipykernel_75600/3769393720.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_completo = pd.read_csv('../../raw_data/flights.csv')#,nrows=500000)


In [28]:
features_prelim = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
                     'SCHEDULED_TIME', 'DISTANCE', 'SCHEDULED_ARRIVAL', 'CANCELLED']
cancelation_df2 = df_completo[features_prelim].copy()

cancelation_df2.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED
0,1,1,4,AS,ANC,SEA,205.0,1448,430,0
1,1,1,4,AA,LAX,PBI,280.0,2330,750,0
2,1,1,4,US,SFO,CLT,286.0,2296,806,0
3,1,1,4,AA,LAX,MIA,285.0,2342,805,0
4,1,1,4,AS,SEA,ANC,235.0,1448,320,0


In [34]:
# Filtra el dataset para tener solo los datos donde CANCELLED es igual a 1
only_cancelled = cancelation_df2.loc[cancelation_df2['CANCELLED'] == 1]

# Muestra el nuevo dataframe filtrado
only_cancelled.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED
32,1,1,4,AS,ANC,SEA,205.0,1448,600,1
42,1,1,4,AA,PHX,DFW,120.0,868,500,1
68,1,1,4,OO,MAF,IAH,87.0,429,637,1
82,1,1,4,MQ,SGF,DFW,95.0,364,700,1
90,1,1,4,OO,RDD,SFO,90.0,199,700,1


In [35]:
only_cancelled.CANCELLED.value_counts()

CANCELLED
1    89884
Name: count, dtype: int64

In [37]:
only_cancelled.to_csv('../../raw_data/ony_cancelled_df.csv', index=False)

In [49]:
only_cancelled.ORIGIN_AIRPORT.unique()

array(['ANC', 'PHX', 'MAF', 'SGF', 'RDD', 'CHS', 'SMX', 'ABI', 'XNA',
       'DCA', 'LGA', 'MCO', 'SFO', 'ONT', 'MSO', 'ORD', 'AMA', 'CLL',
       'HOU', 'DFW', 'JLN', 'CID', 'MLI', 'GSP', 'MHK', 'MOB', 'TLH',
       'CVG', 'CAE', 'TXK', 'CMX', 'SMF', 'LRD', 'MFE', 'FSM', 'AEX',
       'SPS', 'ACT', 'SJT', 'JAN', 'BNA', 'DTW', 'FAR', 'MSN', 'SJC',
       'SAV', 'GRI', 'LAX', 'ATL', 'LEX', 'VPS', 'GPT', 'MGM', 'MSY',
       'SPI', 'IAH', 'PIA', 'ROW', 'FWA', 'FLL', 'HOB', 'COU', 'SAF',
       'GRR', 'LFT', 'TUL', 'ELP', 'DVL', 'TUS', 'GRK', 'PNS', 'GJT',
       'BMI', 'RKS', 'MIA', 'DAL', 'LCH', 'GCK', 'GSO', 'AUS', 'BTR',
       'ICT', 'FLG', 'STL', 'SAT', 'JMS', 'MSP', 'ROC', 'TPA', 'OMA',
       'SHV', 'TOL', 'CRP', 'OKC', 'LAW', 'DSM', 'LBB', 'LIT', 'SAN',
       'DEN', 'MEM', 'APN', 'CMH', 'HSV', 'SLC', 'MKE', 'BHM', 'TYR',
       'MLU', 'GUC', 'ABQ', 'DRO', 'CHA', 'BRO', 'HRL', 'PHL', 'EGE',
       'BWI', 'CLT', 'SDF', 'FCA', 'DIK', 'PUB', 'ASE', 'JFK', 'ISN',
       'BOI', 'CPR',

In [50]:
only_cancelled.ORIGIN_AIRPORT.value_counts()

ORIGIN_AIRPORT
ORD      8548
DFW      6254
LGA      4531
EWR      3110
BOS      2654
         ... 
14689       1
15070       1
12954       1
10158       1
11042       1
Name: count, Length: 609, dtype: int64

### YA TOMAMOS LOS 89.884 FILAS CON VUELOS CANCELADOS, AHORA TOMAREMOS 808.956 (9 VECES LA CANTIDAD DE VUELOS CANCELADOS)

In [38]:
# Filtra el dataset original para tener solo las filas donde CANCELLED sea igual a 0
df_filtered = df_completo[df_completo['CANCELLED'] == 0]

# Toma una muestra aleatoria de 808,956 filas del DataFrame filtrado
sample_not_cancelled_df = df_filtered.sample(n=808956, random_state=42)

# Guarda el nuevo DataFrame como archivo
sample_not_cancelled_df.to_csv('../../raw_data/not_cancelled_df.csv', index=False)


### FINALMENTE, MEZCLAMOS AMBOS DATAFRAMES

In [39]:
# Combinar los dataframes sample_not_cancelled_df y only_cancelled
combined_df = pd.concat([sample_not_cancelled_df, only_cancelled])


In [40]:
combined_df.to_csv('../../raw_data/pre_cleaned_flights.csv', index=False)

# ONE HOT ENCODE EN DATAFRAME COMBINADO

In [41]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse = False) 

# Fit encoder
ohe.fit(combined_df[['AIRLINE']]) 

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")


The categories detected by the OneHotEncoder are [array(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'MQ', 'NK', 'OO', 'UA',
       'US', 'VX', 'WN'], dtype=object)]


/Users/diego/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [43]:
# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "Street" column
combined_df[ohe.get_feature_names_out()] = ohe.transform(combined_df[['AIRLINE']])

# Drop the column "Street" which has been encoded
combined_df.drop(columns = ["AIRLINE"], inplace = True)

# Show the dataset
combined_df.head(3)


The column names for the encoded values are ['AIRLINE_AA' 'AIRLINE_AS' 'AIRLINE_B6' 'AIRLINE_DL' 'AIRLINE_EV'
 'AIRLINE_F9' 'AIRLINE_HA' 'AIRLINE_MQ' 'AIRLINE_NK' 'AIRLINE_OO'
 'AIRLINE_UA' 'AIRLINE_US' 'AIRLINE_VX' 'AIRLINE_WN']


,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
3679572,2015.0,8,17,1,3558.0,N656MQ,ABI,DFW,707.0,701.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5202464,2015.0,11,21,6,1436.0,N367NW,DTW,BWI,1935.0,1934.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
381725,2015.0,1,26,1,1097.0,N3749D,RDU,LAX,759.0,754.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
combined_df.to_csv('../../raw_data/cleaned_flights.csv', index=False)

In [47]:
print(f"The unique values for 'Airlines' are {combined_df.ORIGIN_AIRPORT.unique()}")

The unique values for 'Airlines' are ['ABI' 'DTW' 'RDU' 'ORD' 'BTR' 'FLL' 'MSP' 'SFO' 'DEN' 'PHX' 'ROC' 14307
 14747 'BWI' 'TVC' 'SLC' 'ABQ' 'IAH' 12264 'CLT' 'JNU' 'BOS' 'MKE' 'BNA'
 'ATL' 'EWR' 'LAX' 14100 'GEG' 'PBI' 'MLI' 14122 'SEA' 'TUS' 13303 'JAC'
 'DFW' 'PDX' 'CLE' 'SMF' 'DCA' 'MDW' 'HNL' 'OMA' 'MSY' 14908 'CAE' 'HSV'
 'TPA' 'SJC' 'SAN' 'JFK' 'SJU' 'MCO' 'OAK' 'LGA' 'AUS' 'BUF' 'HOU' 'BZN'
 'GRR' 'MIA' 15370 'LAS' 'IND' 'GJT' 13930 14570 'CLD' 'ALB' 15919 'TYR'
 'PSP' 'EVV' 'VPS' 'AZO' 'FWA' 'STL' 11278 10408 '13367' 14635 'ANC' 'JAN'
 'LGB' 'XNA' 'AEX' '10821' 'PWM' 13487 'FNT' 'PHL' 'JAX' 'HYS' 'DBQ' 12889
 'BRO' 'ONT' 'CHO' 13204 'BDL' 'EGE' 'SNA' 'CHS' 14869 'CVG' 11697 12892
 14107 15304 'DAL' 'OKC' 'ELP' 'PIA' 'CMH' 'BTM' 11433 'SBA' 12953 'MEM'
 'SAV' 'SAT' 12173 'ECP' 'MAF' 'IAD' 14027 14679 12945 'WRG' 'LBE' 'MCI'
 'LFT' 'FSM' 'OGG' 'PVD' 'HPN' 'CRP' 13277 11618 14524 'HLN' 'COS' 'RIC'
 11292 'TYS' 'ISN' 'ASE' 'MSN' 'DAY' 14771 'DSM' 'SDF' 13232 'CHA' 13796
 12884 'MY

In [48]:
combined_df.ORIGIN_AIRPORT.value_counts()

ORIGIN_AIRPORT
ATL      51241
ORD      47342
DFW      39228
LAX      29654
DEN      29369
         ...  
11503        1
11146        1
13184        1
10627        1
13933        1
Name: count, Length: 865, dtype: int64

In [46]:
# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse = False) 

# Fit encoder
ohe.fit(combined_df[['ORIGIN_AIRPORT']]) 

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")


/Users/diego/.pyenv/versions/3.10.6/envs/flight-predictor/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str']